###  CPA xlsx to Tidydata

In [1]:
from databaker.framework import *
import pandas as pd 

In [2]:
import requests
from cachecontrol import CacheControl
from cachecontrol.caches.file_cache import FileCache
from cachecontrol.heuristics import LastModified
from pathlib import Path

session = CacheControl(requests.Session(),
                       cache=FileCache('.cache'),
                       heuristic=LastModified())

sourceFolder = Path('in')
sourceFolder.mkdir(exist_ok=True)

inputURL = 'https://www.ons.gov.uk/file?uri=/businessindustryandtrade/internationaltrade/datasets/uktradeingoodsbyclassificationofproductbyactivity/current/mq10.xlsx'
inputFile = sourceFolder / 'mq10.xlsx'
response = session.get(inputURL)
with open(inputFile, 'wb') as f:
  f.write(response.content)

In [3]:
tab = pd.read_excel(inputFile, header=None)
tab.rename(columns=tab.iloc[1], inplace=True)
tab.rename(columns={'CDID': 'Period'}, inplace=True)
tab

,Period,P25X,P269,P25W,P268,P25Z,P25Y,P25T,P265,P25S,...,P23D,P23C,P23N,P23M,P23P,P23O,P23J,P23I,P23L,P23K
0,Title,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,CPA 08:WW:IM:CP:BOP:NSA: 30.4. Military fighti...,"CPA 08:WW:IM:CP:BOP:NSA: 29. Motor vehicles, t...",CPA 08:WW:IM:CP:BOP:NSA: 30.2. Railway locomot...,CPA 08:WW:IM:CP:BOP:NSA: 29.3. Parts & accesso...,CPA 08:WW:IM:CP:BOP:NSA: 29.2. Bodies for moto...,CPA 08:WW:IM:CP:BOP:NSA: 28.3. Agricultural & ...,CPA 08:WW:IM:CP:BOP:NSA: 30.3A. Aircraft engines,CPA 08:WW:IM:CP:BOP:NSA: 28.2. Other general-p...,...,CPA 08:WW:IM:CP:BOP:NSA: 12.0. Tobacco products,CPA 08:WW:IM:CP:BOP:NSA: 12. Tobacco products,CPA 08:WW:IM:CP:BOP:NSA: 15.1. Tanned & dresse...,CPA 08:WW:IM:CP:BOP:NSA: 15. Leather & related...,CPA 08:WW:IM:CP:BOP:NSA: 16. Wood and prod of ...,CPA 08:WW:IM:CP:BOP:NSA: 15.2. Footwear,CPA 08:WW:IM:CP:BOP:NSA: 14.1. Wearing apparel...,CPA 08:WW:IM:CP:BOP:NSA: 14. Wearing apparel,CPA 08:WW:IM:CP:BOP:NSA: 14.3. Knitted & croch...,CPA 08:WW:IM:CP:BOP:NSA: 14.2. Articles of fur
1,CDID,P25X,P269,P25W,P268,P25Z,P25Y,P25T,P265,P25S,...,P23D,P23C,P23N,P23M,P23P,P23O,P23J,P23I,P23L,P23K
2,PreUnit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Unit,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Release Date,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,...,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018,15-06-2018
5,Next release,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,...,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018,14 September 2018
6,Important Notes,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,1998,16484,103,22961,247,6165,312,482,2567,4664,...,811,811,767,2624,2177,1857,5563,6624,1057,4
8,1999,17408,40,24639,369,6923,308,614,2899,4627,...,1107,1107,785,2806,2234,2021,5869,7006,1132,5
9,2000,16923,22,23772,406,6547,302,557,2722,4902,...,1232,1232,908,2895,2435,1987,6632,7816,1179,5


In [4]:
observations = tab[7:].rename(columns={'CDID': 'Period'})
observations.head()

,Period,P25X,P269,P25W,P268,P25Z,P25Y,P25T,P265,P25S,...,P23D,P23C,P23N,P23M,P23P,P23O,P23J,P23I,P23L,P23K
7,1998,16484,103,22961,247,6165,312,482,2567,4664,...,811,811,767,2624,2177,1857,5563,6624,1057,4
8,1999,17408,40,24639,369,6923,308,614,2899,4627,...,1107,1107,785,2806,2234,2021,5869,7006,1132,5
9,2000,16923,22,23772,406,6547,302,557,2722,4902,...,1232,1232,908,2895,2435,1987,6632,7816,1179,5
10,2001,20238,24,27050,356,6433,379,582,3023,4971,...,1250,1250,952,3194,2501,2242,6966,8313,1341,6
11,2002,21861,20,29407,368,7114,432,714,3132,4883,...,1182,1182,936,3307,2678,2371,7443,8794,1342,9


In [5]:
new_table = pd.melt(observations, id_vars=['Period'], var_name='CDID', value_name='OBS')
new_table.dropna(inplace=True)
new_table.reset_index(drop=True, inplace=True)
print(len(new_table))
new_table.tail(5)
new_table.head(5)

439923


,Period,CDID,OBS
0,1998,P25X,16484
1,1999,P25X,17408
2,2000,P25X,16923
3,2001,P25X,20238
4,2002,P25X,21861


In [6]:
new_table.count()

Period    439923
CDID      439923
OBS       439923
dtype: int64

In [7]:
new_table['Title'] = tab.iloc[0][new_table['CDID']].values
new_table.head()

,Period,CDID,OBS,Title
0,1998,P25X,16484,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
1,1999,P25X,17408,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
2,2000,P25X,16923,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
3,2001,P25X,20238,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
4,2002,P25X,21861,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles


In [8]:
new_table['Period'].unique()

array(['1998', '1999', '2000', '2001', '2002', '2003', '2004', '2005',
       '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013',
       '2014', '2015', '2016', '2017', '1998 Q1', '1998 Q2', '1998 Q3',
       '1998 Q4', '1999 Q1', '1999 Q2', '1999 Q3', '1999 Q4', '2000 Q1',
       '2000 Q2', '2000 Q3', '2000 Q4', '2001 Q1', '2001 Q2', '2001 Q3',
       '2001 Q4', '2002 Q1', '2002 Q2', '2002 Q3', '2002 Q4', '2003 Q1',
       '2003 Q2', '2003 Q3', '2003 Q4', '2004 Q1', '2004 Q2', '2004 Q3',
       '2004 Q4', '2005 Q1', '2005 Q2', '2005 Q3', '2005 Q4', '2006 Q1',
       '2006 Q2', '2006 Q3', '2006 Q4', '2007 Q1', '2007 Q2', '2007 Q3',
       '2007 Q4', '2008 Q1', '2008 Q2', '2008 Q3', '2008 Q4', '2009 Q1',
       '2009 Q2', '2009 Q3', '2009 Q4', '2010 Q1', '2010 Q2', '2010 Q3',
       '2010 Q4', '2011 Q1', '2011 Q2', '2011 Q3', '2011 Q4', '2012 Q1',
       '2012 Q2', '2012 Q3', '2012 Q4', '2013 Q1', '2013 Q2', '2013 Q3',
       '2013 Q4', '2014 Q1', '2014 Q2', '2014 Q3', '2014

In [9]:
import re
YEAR_RE = re.compile(r'[0-9]{4}')
YEAR_MONTH_RE = re.compile(r'([0-9]{4})\s+(JAN|FEB|MAR|APR|MAY|JUN|JUL|AUG|SEP|OCT|NOV|DEC)')
YEAR_QUARTER_RE = re.compile(r'([0-9]{4})\s+(Q[1-4])')

# from https://stackoverflow.com/questions/597476/how-to-concisely-cascade-through-multiple-regex-statements-in-python
class Re(object):
  def __init__(self):
    self.last_match = None
  def fullmatch(self,pattern,text):
    self.last_match = re.fullmatch(pattern,text)
    return self.last_match

def time2period(t):
    gre = Re()
    if gre.fullmatch(YEAR_RE, t):
        return f"year/{t}"
    elif gre.fullmatch(YEAR_MONTH_RE, t):
        year, month = gre.last_match.groups()
        month_num = {'JAN': '01', 'FEB': '02', 'MAR': '03', 'APR': '04', 'MAY': '05', 'JUN': '06',
                     'JUL': '07', 'AUG': '08', 'SEP': '09', 'OCT': '10', 'NOV': '11', 'DEC': '12'}.get(month)
        return f"month/{year}-{month_num}"
    elif gre.fullmatch(YEAR_QUARTER_RE, t):
        year, quarter = gre.last_match.groups()
        return f"quarter/{year}-{quarter}"
    else:
        print(f"no match for {t}")

new_table['Period'] = new_table['Period'].apply(time2period)
new_table

,Period,CDID,OBS,Title
0,year/1998,P25X,16484,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
1,year/1999,P25X,17408,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
2,year/2000,P25X,16923,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
3,year/2001,P25X,20238,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
4,year/2002,P25X,21861,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
5,year/2003,P25X,22729,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
6,year/2004,P25X,23219,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
7,year/2005,P25X,23850,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
8,year/2006,P25X,23994,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles
9,year/2007,P25X,27037,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles


In [10]:
new_table['Geography'] = 'K02000001'
new_table['Unit'] = '£ Million'
new_table['Measure Type'] = 'GBP Total'
new_table.tail(5)

,Period,CDID,OBS,Title,Geography,Unit,Measure Type
439918,quarter/2017-Q1,P23K,8,CPA 08:WW:IM:CP:BOP:NSA: 14.2. Articles of fur,K02000001,£ Million,GBP Total
439919,quarter/2017-Q2,P23K,8,CPA 08:WW:IM:CP:BOP:NSA: 14.2. Articles of fur,K02000001,£ Million,GBP Total
439920,quarter/2017-Q3,P23K,16,CPA 08:WW:IM:CP:BOP:NSA: 14.2. Articles of fur,K02000001,£ Million,GBP Total
439921,quarter/2017-Q4,P23K,18,CPA 08:WW:IM:CP:BOP:NSA: 14.2. Articles of fur,K02000001,£ Million,GBP Total
439922,quarter/2018-Q1,P23K,9,CPA 08:WW:IM:CP:BOP:NSA: 14.2. Articles of fur,K02000001,£ Million,GBP Total


In [11]:
cord_sitc_classification_table_url = 'https://drive.google.com/uc?export=download&id=1uJck_DtSgLs0XcEuKDB0swzj1UrWmauj'
temp_table = pd.read_excel(cord_sitc_classification_table_url, sheet_name = 0)

In [12]:
temp_table = temp_table.reset_index()

In [13]:
temp_table.drop(['level_0','level_1','Sequence'], axis =1, inplace = True)

In [14]:
temp_table.tail()

,cdid,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
6196,BPFR,5min8minE,WW,IM,BOP,IDEF,NSA,M
6197,BPGR,5min8minE,WW,IM,BOP,IDEF,SA,M
6198,BPCR,5min8minE,WW,IM,BOP,VM,NSA,M
6199,ELAI,5min8minE,WW,IM,BOP,VM,SA,M
6200,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:
CSDB_classification_tables_url = 'https://drive.google.com/uc?export=download&id=1miAzQ6s8om4Ark3BpRk3Y90OAWfWErTb'
classification1 = pd.read_excel(CSDB_classification_tables_url, 'cord_sitc')

In [16]:
classification1.tail(5)

,cdid,COMMODITY,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
6193,ELAX,5min8minE,WW,IM,BOP,DEF,NSA,Q
6194,BPFR,5min8minE,WW,IM,BOP,IDEF,NSA,Q
6195,BPGR,5min8minE,WW,IM,BOP,IDEF,SA,Q
6196,BPCR,5min8minE,WW,IM,BOP,VM,NSA,Q
6197,ELAI,5min8minE,WW,IM,BOP,VM,SA,Q


In [17]:
classification2 = pd.read_excel(CSDB_classification_tables_url, 'cord_cpa')

In [18]:
classification2.head(5)

,cdid,PRODUCT,AREA,DIRECTION,BASIS,PRICE,SEASADJ,PERIOD
0,P42L,24.2,EU,EX,BOP,CP,NSA,Q
1,P483,24.2,EU,EX,BOP,CP,SA,Q
2,P4DJ,24.2,EU,EX,BOP,CVM,NSA,Q
3,P4IZ,24.2,EU,EX,BOP,CVM,SA,Q
4,P3EP,24.2,EU,IM,BOP,CP,NSA,Q


In [19]:
classification2.rename(index = str, columns = {'PRODUCT':'COMMODITY'}, inplace = True)

In [20]:
classification3 = pd.read_excel(CSDB_classification_tables_url, 'cord_country')

In [21]:
classification3.head(5)

,cdid,COUNTRY,DIRECTION,BASIS,SEASADJ,PERIOD
0,KN2O,XS,BAL,BOP,NSA,Q
1,LGDS,V4,BAL,BOP,NSA,Q
2,L87P,V3,BAL,BOP,NSA,Q
3,L87J,V2,BAL,BOP,NSA,Q
4,MHN8,I7,BAL,BOP,NSA,Q


In [22]:
classification3['COMMODITY'] = ''
classification3['PRICE'] = ''

In [23]:
classification3.rename(index = str, columns = {'COUNTRY':'AREA'}, inplace = True)

In [24]:
temp_table = pd.concat([temp_table, classification1, classification2,classification3])

In [25]:
temp_table.head()

,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ,cdid
0,EU,BOP,2plus4,BAL,M,CP,NSA,SDSX
1,EU,BOP,5minus8,BAL,M,CP,NSA,SGLO
2,EU,BOP,7plus8,BAL,M,CP,NSA,SFJC
3,EU,BOP,TminusO,BAL,M,CP,NSA,LKTX
4,EU,BOP,0plus1,BAL,M,CP,NSA,SDMS


In [26]:
temp_table.shape

(18264, 8)

In [27]:
temp_table.drop_duplicates(['cdid'], keep='first', inplace=True)

In [28]:
temp_table.shape

(12043, 8)

In [29]:
new_table.head()

,Period,CDID,OBS,Title,Geography,Unit,Measure Type
0,year/1998,P25X,16484,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total
1,year/1999,P25X,17408,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total
2,year/2000,P25X,16923,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total
3,year/2001,P25X,20238,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total
4,year/2002,P25X,21861,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total


In [30]:
# new_table.head(1)['Cdid']

In [31]:
# temp_table.head(1)['cdid']

In [32]:
new_table = pd.merge(new_table,temp_table, how = 'left', left_on = 'CDID', right_on = 'cdid')

In [33]:
new_table.head(5)

,Period,CDID,OBS,Title,Geography,Unit,Measure Type,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ,cdid
0,year/1998,P25X,16484,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total,WW,BOP,29.1,IM,Q,CP,NSA,P25X
1,year/1999,P25X,17408,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total,WW,BOP,29.1,IM,Q,CP,NSA,P25X
2,year/2000,P25X,16923,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total,WW,BOP,29.1,IM,Q,CP,NSA,P25X
3,year/2001,P25X,20238,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total,WW,BOP,29.1,IM,Q,CP,NSA,P25X
4,year/2002,P25X,21861,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total,WW,BOP,29.1,IM,Q,CP,NSA,P25X


In [34]:
new_table.drop(['cdid'], axis = 1, inplace = True)

In [35]:
new_table.head(2)

,Period,CDID,OBS,Title,Geography,Unit,Measure Type,AREA,BASIS,COMMODITY,DIRECTION,PERIOD,PRICE,SEASADJ
0,year/1998,P25X,16484,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total,WW,BOP,29.1,IM,Q,CP,NSA
1,year/1999,P25X,17408,CPA 08:WW:IM:CP:BOP:NSA: 29.1. Motor vehicles,K02000001,£ Million,GBP Total,WW,BOP,29.1,IM,Q,CP,NSA


In [36]:
new_table['ONS Partner Geography'] ='legacy/'+ new_table['AREA'].astype(str)

In [37]:
new_table.rename(index= str, columns= {'OBS':'Value', 'DIRECTION':'Flow',
                                       'COMMODITY':'CPA 2008', 'SEASADJ':'Seasonal Adjustment',
                                       'PRICE':'Price Classification', 'BASIS': 'International Trade Basis'
                                      }, inplace = True)

In [38]:
new_table = new_table[new_table['Value'] != 0 ]

In [39]:
new_table = new_table[['ONS Partner Geography','Period','CDID','International Trade Basis','CPA 2008','Seasonal Adjustment','Flow','Price Classification','Measure Type','Value','Unit']]

In [40]:
new_table.tail(5)

,ONS Partner Geography,Period,CDID,International Trade Basis,CPA 2008,Seasonal Adjustment,Flow,Price Classification,Measure Type,Value,Unit
439918,legacy/WW,quarter/2017-Q1,P23K,BOP,14.2,NSA,IM,CP,GBP Total,8,£ Million
439919,legacy/WW,quarter/2017-Q2,P23K,BOP,14.2,NSA,IM,CP,GBP Total,8,£ Million
439920,legacy/WW,quarter/2017-Q3,P23K,BOP,14.2,NSA,IM,CP,GBP Total,16,£ Million
439921,legacy/WW,quarter/2017-Q4,P23K,BOP,14.2,NSA,IM,CP,GBP Total,18,£ Million
439922,legacy/WW,quarter/2018-Q1,P23K,BOP,14.2,NSA,IM,CP,GBP Total,9,£ Million


In [41]:
new_table.fillna('NA', inplace = True)

In [42]:
new_table['Price Classification'].unique()

array(['CP', 'CVM'], dtype=object)

In [43]:
new_table.Flow.unique()

array(['IM', 'EX'], dtype=object)

In [44]:
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('EX', 'Exports'))
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('IM', 'Imports'))
new_table['Flow'] = new_table['Flow'].map(lambda cell:cell.replace('BAL', 'Balance'))

In [45]:
new_table['ONS Partner Geography'].unique()

array(['legacy/WW', 'legacy/EU', 'legacy/RW'], dtype=object)

In [46]:
new_table['International Trade Basis'].unique()

array(['BOP'], dtype=object)

CPA codes: the first number should be padded with a leading zero if it's < 10.

In [47]:
import re
r = re.compile('([1-9])\.')

new_table['CPA 2008'] = new_table['CPA 2008'].map(lambda x: '0' + str(x) if r.match(str(x)) else str(x))

In [48]:
str(new_table.head(1)['CPA 2008'])[-1]

't'

In [49]:
new_table.iloc[7200]

ONS Partner Geography              legacy/WW
Period                       quarter/2008-Q2
CDID                                    P287
International Trade Basis                BOP
CPA 2008                                08.1
Seasonal Adjustment                       SA
Flow                                 Imports
Price Classification                      CP
Measure Type                       GBP Total
Value                                     69
Unit                               £ Million
Name: 7535, dtype: object

In [50]:
def func(x):
    try:
        if x == 'TOTAL':
            return 'TOTAL'
        elif (x == 'A')|(x == 'B')|(x == 'C')|(x == 'D')|(x == 'E')|(x == 'J')|(x == 'M')|(x == 'R')|(x == 'S'):
            return 'section/'+ str(x)     
        elif str(x)[-2] == '.':
            return 'group/' + str(x) 
        elif  '.' in str(x) :
             return 'class/' + str(x)
        else:
            try:
                str(x)[-2]
                return 'division/' + str(x)
            except:
                return 'division/0' + str(x)
    except:
            try:
                str(x)[-2]
                return 'division/' + str(x)
            except:
                return 'division/0' + str(x)
   

new_table['CPA 2008'] = new_table.apply(lambda row: func(row['CPA 2008']), axis = 1)

In [51]:
new_table.iloc[201]

ONS Partner Geography              legacy/WW
Period                       quarter/2018-Q1
CDID                                    P269
International Trade Basis                BOP
CPA 2008                          group/30.4
Seasonal Adjustment                      NSA
Flow                                 Imports
Price Classification                      CP
Measure Type                       GBP Total
Value                                     34
Unit                               £ Million
Name: 201, dtype: object

In [52]:
new_table.tail()

,ONS Partner Geography,Period,CDID,International Trade Basis,CPA 2008,Seasonal Adjustment,Flow,Price Classification,Measure Type,Value,Unit
439918,legacy/WW,quarter/2017-Q1,P23K,BOP,group/14.2,NSA,Imports,CP,GBP Total,8,£ Million
439919,legacy/WW,quarter/2017-Q2,P23K,BOP,group/14.2,NSA,Imports,CP,GBP Total,8,£ Million
439920,legacy/WW,quarter/2017-Q3,P23K,BOP,group/14.2,NSA,Imports,CP,GBP Total,16,£ Million
439921,legacy/WW,quarter/2017-Q4,P23K,BOP,group/14.2,NSA,Imports,CP,GBP Total,18,£ Million
439922,legacy/WW,quarter/2018-Q1,P23K,BOP,group/14.2,NSA,Imports,CP,GBP Total,9,£ Million


In [53]:
new_table['CPA 2008'].unique()

array(['group/29.1', 'group/30.4', 'division/29', 'group/30.2',
       'group/29.3', 'group/29.2', 'group/28.3', 'class/30.3A',
       'group/28.2', 'group/30.3', 'group/28.9', 'class/30.3C',
       'group/28.4', 'class/30.3B', 'group/30.9', 'division/32',
       'division/31', 'group/32.9', 'group/32.5', 'division/35',
       'section/D', 'group/32.2', 'group/32.1', 'group/32.4', 'group/32.3',
       'division/38', 'group/59.2', 'group/38.2', 'division/71',
       'group/38.1', 'section/M', 'group/35.1', 'section/E', 'group/24.1',
       'group/25.7', 'division/24', 'group/25.4', 'group/24.3',
       'group/24.2', 'group/25.9', 'group/23.6', 'group/25.1',
       'group/23.5', 'division/25', 'group/23.9', 'group/25.3',
       'group/23.7', 'group/25.2', 'group/26.1', 'division/26',
       'group/24.4', 'group/26.7', 'group/26.6', 'division/27',
       'group/26.8', 'group/26.3', 'group/26.2', 'group/26.5',
       'group/26.4', 'group/27.9', 'group/27.5', 'group/28.1',
       'group/30.

In [54]:
productlist = new_table['CPA 2008'].unique()

In [55]:
pl = pd.Series(productlist)

In [56]:
pl.to_csv('Productlist.csv', index = False)

In [57]:
new_table.dtypes

ONS Partner Geography        object
Period                       object
CDID                         object
International Trade Basis    object
CPA 2008                     object
Seasonal Adjustment          object
Flow                         object
Price Classification         object
Measure Type                 object
Value                         int64
Unit                         object
dtype: object

In [58]:
import numpy as np

destFolder = Path('out')
destFolder.mkdir(exist_ok=True, parents=True)

sliceSize = 50000
for i in np.arange(len(new_table)//sliceSize):
    destFile = destFolder / f'observations_{i:04}.csv'
    new_table.iloc[i*sliceSize:i*sliceSize+sliceSize-1].to_csv(destFile, index=False)